In [2]:
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.callbacks import Callback
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import keras
import subprocess
import os

import wandb
from wandb.keras import WandbCallback

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [4]:
def load_fer2013():
    if not os.path.exists("fer2013"):
        print("Downloading the face emotion dataset...")
        subprocess.check_output("curl -SL https://www.dropbox.com/s/opuvvdv3uligypx/fer2013.tar | tar xz", shell=True)
    data = pd.read_csv("fer2013/fer2013.csv")
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = np.asarray(pixel_sequence.split(' '), dtype=np.uint8).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), (width, height))
        faces.append(face.astype('float32'))

    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()

    val_faces = faces[int(len(faces) * 0.8):]
    val_emotions = emotions[int(len(faces) * 0.8):]
    train_faces = faces[:int(len(faces) * 0.8)]
    train_emotions = emotions[:int(len(faces) * 0.8)]
    
    return train_faces, train_emotions, val_faces, val_emotions


In [5]:
# loading dataset

train_faces, train_emotions, val_faces, val_emotions = load_fer2013()
num_samples, num_classes = train_emotions.shape

train_faces /= 255.
val_faces /= 255.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [6]:
train_faces.shape

(28709, 48, 48, 1)

In [7]:
datagen = ImageDataGenerator(
        #rotation_range=20,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #rescale=1./255,
        #shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
datagen.fit(train_faces, augment=True, rounds=2)

In [9]:
run = wandb.init()
input_shape = (48, 48, 1)
config = run.config

#config.first_layer_convs = 32
#config.first_layer_conv_width = 3
#config.first_layer_conv_height = 3
#config.dropout = 0.2
#config.dense_layer_size = 128
#config.img_width = 28
#config.img_height = 28
config.num_epochs = 32

W&B Run: https://app.wandb.ai/qualcomm/emotion-aug21/runs/y0osny07
Wrap your training loop with `with wandb.monitor():` to display live results.


In [20]:
model = Sequential()

model.add(Conv2D(32, (3,3), 
    #(config.first_layer_conv_width, config.first_layer_conv_height),
    input_shape=(48, 48,1),
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3,3), 
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), 
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(8*64*7, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

opt = keras.optimizers.Adam(lr=1*0.001) #, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss='categorical_crossentropy',
metrics=['accuracy'])

#model.load_weights("emotion.h5")

#model.fit(train_faces, train_emotions, batch_size=32,
#        epochs=config.num_epochs, verbose=1, callbacks=[
#            WandbCallback(data_type="image", labels=["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"])
#        ], validation_data=(val_faces, val_emotions))


train_generator = datagen.flow(train_faces, train_emotions, batch_size=32)
val_generator = datagen.flow(val_faces, val_emotions, batch_size=32)
model.fit_generator(train_generator,
            steps_per_epoch=len(train_faces)//32, 
            epochs=config.num_epochs, verbose=1,
            callbacks=[
                WandbCallback(data_type="image", labels=["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"], generator=val_generator)
            ],
            validation_data=val_generator,
            validation_steps=len(val_faces)//32)



Epoch 1/32
897/897 [==============================] - 15s 17ms/step - loss: 1.7172 - acc: 0.3053 - val_loss: 1.5335 - val_acc: 0.4140
Epoch 2/32
897/897 [==============================] - 14s 16ms/step - loss: 1.5206 - acc: 0.4109 - val_loss: 1.3938 - val_acc: 0.4643
Epoch 3/32
897/897 [==============================] - 14s 15ms/step - loss: 1.4330 - acc: 0.4444 - val_loss: 1.3566 - val_acc: 0.4819
Epoch 4/32
897/897 [==============================] - 14s 15ms/step - loss: 1.3761 - acc: 0.4708 - val_loss: 1.2757 - val_acc: 0.5116
Epoch 5/32
897/897 [==============================] - 13s 15ms/step - loss: 1.3434 - acc: 0.4834 - val_loss: 1.2542 - val_acc: 0.5223
Epoch 6/32
897/897 [==============================] - 14s 15ms/step - loss: 1.3184 - acc: 0.4939 - val_loss: 1.2230 - val_acc: 0.5281
Epoch 7/32
897/897 [==============================] - 14s 15ms/step - loss: 1.2903 - acc: 0.5070 - val_loss: 1.2433 - val_acc: 0.5203
Epoch 8/32
897/897 [==============================] - 14s 15ms

In [16]:
model.save("emotion.h5")